Importation des librairies/modules

In [31]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as nu
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('data/train.csv')

Traitements des données </br>
- Repérage des données manquantes
- Création d'une colonne 'Title' pour stocker les titres extraits de la colonne 'Name'
- Création d'une fonction impute_age pour affecter un âge (médiane des âges selon le titre et Pclass)
- Encodage des données dont les valeurs ne sont pas numériques 
- Remplissage de colonne 'Fare' à partir de la médiane des valeurs de la colonne (choix de simplicité)

In [32]:
# Extraction titre
df['Title'] = df['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False).str.strip()

# Remplissage des âges manquants par médiane par groupe
age_medians = df.groupby(['Title', 'Pclass'])['Age'].median()
def impute_age(row):
    if pd.isnull(row['Age']):
        return age_medians.get((row['Title'], row['Pclass']), df['Age'].median())
    return row['Age']
df['Age'] = df.apply(impute_age, axis=1)

# Encodage des variables catégorielles
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Title'] = le.fit_transform(df['Title'])
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Embarked'] = le.fit_transform(df['Embarked'])

# Remplissage de 'Fare'
df['Fare'] = df['Fare'].fillna(df['Fare'].median())

# Suppression des colonnes inutiles
df.drop(columns=['Cabin', 'Ticket', 'Name'], inplace=True)


Visualisation

I - Répartition des survivants

In [38]:
fig = px.histogram(df, x='Survived',
                   color=df['Survived'].map({0: 'Mort', 1: 'Survécu'}),
                   labels={'Survived': 'Survie'},
                   category_orders={'Survived': [0,1]},
                   
                   )
fig.update_layout(xaxis_tickvals=[0, 1], xaxis_ticktext=['Mort', 'Survivant'])
fig.show()

II - Taux de survie par sexe

In [42]:
fig = px.histogram(df,
                   x=df['Sex'].map({0: 'Femme', 1: 'Homme'}),
                   color=df['Survived'].map({0: 'Mort', 1: 'Survécu'}),
                   barmode='group',
                   histnorm='percent',
                   text_auto=True,
                   title="Taux de survie selon le sexe"
                   )
fig.show()

III - Taux de survie par classe de passager

In [41]:
fig = px.histogram(df,
                   x='Pclass',
                   color=df['Survived'].map({0: 'Mort', 1: 'Survécu'}),
                   title='Taux de survie selon la classe de passager',
                   barmode='group',
                   histnorm='percent',
                   text_auto=True
                   )
fig.update_layout(xaxis_tickvals=[0, 1, 2, 3])
fig.show()

IV - Taux de survie selon l'âge

In [36]:
fig = px.histogram(df,
                   x='Age',
                   color=df['Survived'].map({0: 'Mort', 1: 'Survécu'}),
                   barmode='group',
                   title='Taux de survie selon l\'âge',
                   histnorm='percent',
                   text_auto=True)
fig.show()